In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#Connect to data folder of NerLTR-DTA
root_path = '/content/gdrive/My Drive/KI JKU/Seminar_PracticalW_MasterThesis/MasterThesis/NerLTR/data/'  
import os
os.chdir(root_path)
!pwd

/content/gdrive/My Drive/KI JKU/Seminar_PracticalW_MasterThesis/MasterThesis/NerLTR/data


In [3]:
import numpy as np
import pandas as pd
import linecache
import math
from collections import Counter
import pickle
from posixpath import split

#Load Data

In [4]:
path_test_KIBA = 'RankLibEval/data_sim/test_KIBA.txt'

In [5]:
#split1
path_split1_score = 'RankLibEval/data_sim/split1/val_score_1.txt'
path_split1_val = 'RankLibEval/data_sim/split1/val1.txt'
path_split1_test_score = 'RankLibEval/data_sim/split1/test_score_1.txt'
#split2
path_split2_score = 'RankLibEval/data_sim/split2/val_score_2.txt'
path_split2_val = 'RankLibEval/data_sim/split2/val2.txt'
path_split2_test_score = 'RankLibEval/data_sim/split2/test_score_2.txt'
#split1
path_split3_score = 'RankLibEval/data_sim/split3/val_score_3.txt'
path_split3_val = 'RankLibEval/data_sim/split3/val3.txt'
path_split3_test_score = 'RankLibEval/data_sim/split3/test_score_3.txt'
#split1
path_split4_score = 'RankLibEval/data_sim/split4/val_score_4.txt'
path_split4_val = 'RankLibEval/data_sim/split4/val4.txt'
path_split4_test_score = 'RankLibEval/data_sim/split4/test_score_4.txt'
#split1
path_split5_score = 'RankLibEval/data_sim/split5/val_score_5.txt'
path_split5_val = 'RankLibEval/data_sim/split5/val5.txt'
path_split5_test_score = 'RankLibEval/data_sim/split5/test_score_5.txt'
#full
path_full_score = 'RankLibEval/data_sim/full/test_score.txt'

##Get Predictions

In [6]:
def get_pred_score(input_path):
  pred = []
  with open(input_path) as f:
    for line in f:
      split=line.split()
      pred_score = float(split[2])
      pred.append(pred_score)
  return pred

#Val
pred_split1_val = get_pred_score(path_split1_score)
pred_split2_val = get_pred_score(path_split2_score)
pred_split3_val = get_pred_score(path_split3_score)
pred_split4_val = get_pred_score(path_split4_score)
pred_split5_val = get_pred_score(path_split5_score)

#Test
pred_split1_test = get_pred_score(path_split1_test_score)
pred_split2_test = get_pred_score(path_split2_test_score)
pred_split3_test = get_pred_score(path_split3_test_score)
pred_split4_test = get_pred_score(path_split4_test_score)
pred_split5_test = get_pred_score(path_split5_test_score)

In [7]:
#full
pred_full_test = get_pred_score(path_full_score)

In [8]:
print(f'Len Prd Split1: {len(pred_split1_val)}')

Len Prd Split1: 19709


In [9]:
#Val
print(pred_split1_val[0])
print(pred_split2_val[0])
print(pred_split3_val[0])
print(pred_split4_val[0])
print(pred_split5_val[0])
#test
print(pred_split1_test[0])
print(pred_split2_test[0])
print(pred_split3_test[0])
print(pred_split4_test[0])
print(pred_split5_test[0])
#full
print(pred_full_test[0])

11.502923011779785
12.542203903198242
11.519950866699219
11.724344253540039
10.174342155456543
13.077567100524902
12.832326889038086
12.937058448791504
12.672809600830078
12.840749740600586
12.76187801361084


##Get Truth

In [10]:
def get_truth(input_path):
  truth = []
  with open(input_path) as f:
    for line in f:
      split=line.split()
      pred_score = float(split[0])
      truth.append(pred_score)
  return truth

#Val
truth_split1_val = get_truth(path_split1_val)
truth_split2_val = get_truth(path_split2_val)
truth_split3_val = get_truth(path_split3_val)
truth_split4_val = get_truth(path_split4_val)
truth_split5_val = get_truth(path_split5_val)
#Test
truth_test = get_truth(path_test_KIBA)

In [11]:
print(f'Len Truth Split1: {len(truth_split1_val)}')

Len Truth Split1: 19709


In [12]:
#Val
print(truth_split1_val[0])
print(truth_split2_val[0])
print(truth_split3_val[0])
print(truth_split4_val[0])
print(truth_split5_val[0])
#Test
print(truth_test[0])

11.200000128
12.599996932
11.300000237
11.599999679
9.798970004
14.400162274


#Get IDs

In [13]:
#Affinities
path_train_fold1_aff_KIBA = 'affi/train_aff_fold1_KIBA.pkl'
path_train_fold2_aff_KIBA = 'affi/train_aff_fold2_KIBA.pkl'
path_train_fold3_aff_KIBA = 'affi/train_aff_fold3_KIBA.pkl'
path_train_fold4_aff_KIBA = 'affi/train_aff_fold4_KIBA.pkl'
path_train_fold5_aff_KIBA = 'affi/train_aff_fold5_KIBA.pkl'
path_test_aff_KIBA = 'affi/test_aff_KIBA.pkl'

In [14]:
#KIBA  fold 1-4 = train; fold 5 = val 
#folds
train_affi_fold1_KIBA = pd.read_pickle(path_train_fold1_aff_KIBA)
y_train_fold1_KIBA_ID = train_affi_fold1_KIBA['target_ID'].tolist()
train_affi_fold2_KIBA = pd.read_pickle(path_train_fold2_aff_KIBA)
y_train_fold2_KIBA_ID = train_affi_fold2_KIBA['target_ID'].tolist()
train_affi_fold3_KIBA = pd.read_pickle(path_train_fold3_aff_KIBA)
y_train_fold3_KIBA_ID = train_affi_fold3_KIBA['target_ID'].tolist()
train_affi_fold4_KIBA = pd.read_pickle(path_train_fold4_aff_KIBA)
y_train_fold4_KIBA_ID = train_affi_fold4_KIBA['target_ID'].tolist()
train_affi_fold5_KIBA = pd.read_pickle(path_train_fold5_aff_KIBA)
y_train_fold5_KIBA_ID = train_affi_fold5_KIBA['target_ID'].tolist()

y_train_KIBA_ID = y_train_fold1_KIBA_ID#+y_train_fold2_KIBA_ID+y_train_fold3_KIBA_ID+y_train_fold4_KIBA_ID+y_train_fold5_KIBA_ID

#test
test_affi_KIBA = pd.read_pickle(path_test_aff_KIBA)
y_test_KIBA_ID = test_affi_KIBA['target_ID'].tolist()

#lengths
print(f'Len Fold1:{len(y_train_fold1_KIBA_ID)}')
print(f'Len Fold2:{len(y_train_fold2_KIBA_ID)}')
print(f'Len Fold3:{len(y_train_fold3_KIBA_ID)}')
print(f'Len Fold4:{len(y_train_fold4_KIBA_ID)}')
print(f'Len Fold5:{len(y_train_fold5_KIBA_ID)}')
print(f'Len Test:{len(y_test_KIBA_ID)}')

Len Fold1:19709
Len Fold2:19709
Len Fold3:19709
Len Fold4:19709
Len Fold5:19709
Len Test:19709


##Metrics

In [15]:
# install lifelines
!pip install -q lifelines
from lifelines.utils import concordance_index
from sklearn.metrics import mean_squared_error, r2_score

     |████████████████████████████████| 349 kB 28.7 MB/s 
     |████████████████████████████████| 77 kB 5.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.1.5 requires typing-extensions<4.2.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.4.0 which is incompatible.
spacy 3.4.2 requires typing-extensions<4.2.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.4.0 which is incompatible.
confection 0.0.3 requires typing-extensions<4.2.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.4.0 which is incompatible.


In [16]:
def r_squared_error(y_obs,y_pred):
    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)
    y_obs_mean = [np.mean(y_obs) for y in y_obs]
    y_pred_mean = [np.mean(y_pred) for y in y_pred]

    mult = sum((y_pred - y_pred_mean) * (y_obs - y_obs_mean))
    mult = mult * mult

    y_obs_sq = sum((y_obs - y_obs_mean)*(y_obs - y_obs_mean))
    y_pred_sq = sum((y_pred - y_pred_mean) * (y_pred - y_pred_mean) )

    return mult / float(y_obs_sq * y_pred_sq)
def squared_error_zero(y_obs,y_pred):
    k = get_k(y_obs,y_pred)

    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)
    y_obs_mean = [np.mean(y_obs) for y in y_obs]
    upp = sum((y_obs - (k*y_pred)) * (y_obs - (k* y_pred)))
    down= sum((y_obs - y_obs_mean)*(y_obs - y_obs_mean))

    return 1 - (upp / float(down))

def get_k(y_obs,y_pred):
    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)

    return sum(y_obs*y_pred) / float(sum(y_pred*y_pred))
def get_rm2(ys_orig,ys_line):
    r2 = r_squared_error(ys_orig, ys_line)
    r02 = squared_error_zero(ys_orig, ys_line)
    return r2 * (1 - np.sqrt(np.absolute((r2*r2)-(r02*r02))))

import torch 
def concordance_index_taskwise(predictions, labels, target_ids):
    CIs = list()
    target_id_list = list()
    predictions = torch.tensor(predictions)
    labels = torch.tensor(labels)
    target_ids = torch.tensor(target_ids)

    for target_idx in torch.unique(target_ids):
        rows = torch.where(target_ids == target_idx)
        preds = predictions[rows]
        y = labels[rows]

        if torch.unique(y).shape[0] >= 2:
            ci = concordance_index(y, preds)
            CIs.append(ci)
            target_id_list.append(target_idx.item())
        else:
            CIs.append(np.nan)
            target_id_list.append(target_idx.item())
    return np.nanmean(CIs), CIs, target_id_list

#Trained on full data with similarities

In [18]:
print("Full")
ci, CIs, _ = concordance_index_taskwise(pred_full_test, truth_test, y_test_KIBA_ID)
print(f'Concordance index New:\t {ci}')
print(f'Concordance index Old:\t {concordance_index(truth_test, pred_full_test)}')
print(f"MSE\t\t：{mean_squared_error(truth_test, pred_full_test)}")
print(f"rm2\t\t：{get_rm2(truth_test, pred_full_test)}")

Full
Concordance index New:	 0.8283999881913411
Concordance index Old:	 0.8456963636662105
MSE		：0.19863767883059186
rm2		：0.6802624019050677


In [19]:
CIs

[0.7117117117117117,
 0.8808630393996247,
 0.8070342205323194,
 0.6263736263736264,
 0.8809828512925518,
 0.8493769011873221,
 0.7549668874172185,
 0.8542291541691661,
 0.9084194977843427,
 0.6817800167926112,
 0.861878453038674,
 0.7839101964452759,
 0.7090522070356262,
 1.0,
 0.7975482912332839,
 0.6666666666666666,
 0.7911400024076081,
 0.8485710010474338,
 0.8692933083176986,
 0.9333333333333333,
 0.8566002117961232,
 0.8072108603667136,
 0.8717948717948718,
 0.7724418801968437,
 0.8215402823569194,
 0.7774638266161584,
 0.8571428571428571,
 0.882104579408899,
 0.8445505356311132,
 0.9547141796585004,
 0.9216867469879518,
 0.8572434430558,
 0.8620442951350006,
 0.8445640847516499,
 0.8853061224489795,
 0.7639939485627837,
 0.8656356580030578,
 1.0,
 0.9126833885470894,
 0.8620976692563818,
 0.7483116256632899,
 0.6715411558669002,
 0.8502139800285307,
 0.7104818258664413,
 0.8267744202389319,
 1.0,
 0.8071533152060238,
 0.7754322225006264,
 0.7788141025641026,
 0.8172043010752689,


#Validation Set

In [ ]:
print("Split1")
print(f'Concordance index:\t {concordance_index(truth_split1_val, pred_split1_val)}')
print(f"MSE\t\t：{mean_squared_error(truth_split1_val, pred_split1_val)}")
print(f"rm2\t\t：{get_rm2(truth_split1_val, pred_split1_val)}")

In [ ]:
print("Split2")
print(f'Concordance index:\t {concordance_index(truth_split2_val, pred_split2_val)}')
print(f"MSE\t\t：{mean_squared_error(truth_split2_val, pred_split2_val)}")
print(f"rm2\t\t：{get_rm2(truth_split2_val, pred_split2_val)}")

In [ ]:
print("Split3")
print(f'Concordance index:\t {concordance_index(truth_split3_val, pred_split3_val)}')
print(f"MSE\t\t：{mean_squared_error(truth_split3_val, pred_split3_val)}")
print(f"rm2\t\t：{get_rm2(truth_split3_val, pred_split3_val)}")

In [ ]:
print("Split4")
print(f'Concordance index:\t {concordance_index(truth_split4_val, pred_split4_val)}')
print(f"MSE\t\t：{mean_squared_error(truth_split4_val, pred_split4_val)}")
print(f"rm2\t\t：{get_rm2(truth_split4_val, pred_split4_val)}")

In [ ]:
print("Split5")
print(f'Concordance index:\t {concordance_index(truth_split5_val, pred_split5_val)}')
print(f"MSE\t\t：{mean_squared_error(truth_split5_val, pred_split5_val)}")
print(f"rm2\t\t：{get_rm2(truth_split5_val, pred_split5_val)}")

#Test Set

In [ ]:
print("Split1")
print(f'Concordance index:\t {concordance_index(truth_test, pred_split1_test)}')
print(f"MSE\t\t：{mean_squared_error(truth_test, pred_split1_test)}")
print(f"rm2\t\t：{get_rm2(truth_test, pred_split1_test)}")

In [ ]:
print("Split2")
print(f'Concordance index:\t {concordance_index(truth_test, pred_split2_test)}')
print(f"MSE\t\t：{mean_squared_error(truth_test, pred_split2_test)}")
print(f"rm2\t\t：{get_rm2(truth_test, pred_split2_test)}")

In [ ]:
print("Split3")
print(f'Concordance index:\t {concordance_index(truth_test, pred_split3_test)}')
print(f"MSE\t\t：{mean_squared_error(truth_test, pred_split3_test)}")
print(f"rm2\t\t：{get_rm2(truth_test, pred_split3_test)}")

In [ ]:
print("Split4")
print(f'Concordance index:\t {concordance_index(truth_test, pred_split4_test)}')
print(f"MSE\t\t：{mean_squared_error(truth_test, pred_split4_test)}")
print(f"rm2\t\t：{get_rm2(truth_test, pred_split4_test)}")

In [ ]:
print("Split5")
print(f'Concordance index:\t {concordance_index(truth_test, pred_split5_test)}')
print(f"MSE\t\t：{mean_squared_error(truth_test, pred_split5_test)}")
print(f"rm2\t\t：{get_rm2(truth_test, pred_split5_test)}")